<a href="https://colab.research.google.com/github/Stupid-Creations/HoLLMes/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import torch

embed_size = 384
block_size = 256
dropout = 0.2
n_layer = 6
n_head = 6
device = "cuda" if torch.cuda.is_available() else "cpu"

#print(device)

In [ ]:
text = open("drive/MyDrive/YAY.txt","r").read()
preprocessed = re.split('([\s])',text)

vocab = sorted(list(set(preprocessed)))
vocab_size = len(vocab)

encode = lambda x: [vocab.index(i) for i in x]
decode = lambda x: ''.join([vocab[i] for i in x])

print(vocab)

print(decode(encode(preprocessed[:100])))

PREFACE

I fear that Mr. Sherlock Holmes may become like one of those popular
tenors who, having outlived their time, are still tempted to make
repeated farewell bows to their indulgent audiences.  This must cease
and he must go the way of all flesh, material or imaginary.  


In [ ]:
print(decode(encode(re.split('([\s])',"He Creeping Cunard"))))
tokenized = torch.tensor(encode(preprocessed))
train_data = tokenized[:int(len(tokenized)*0.9)]
val_data = tokenized[int(len(tokenized)*0.9):]
print(len(train_data))

He Creeping Cunard
161072


In [ ]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (32,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb,yb = get_batch('train')

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class trans_block(nn.Module):
    def __init__(self,embed_size,heads):
        super().__init__()
        head_size = embed_size // heads
        self.attention = Heads(heads,head_size)
        self.ff_layer = FF_Layer(embed_size)
        self.lnorm1 = nn.LayerNorm(embed_size)
        self.lnorm2 = nn.LayerNorm(embed_size)
    def forward(self,x):
        x = x + self.attention(self.lnorm1(x))
        x = x + self.ff_layer(self.lnorm2(x))
        return x

class Head(nn.Module):
    def __init__(self,headsize):
        super().__init__()
        self.key = nn.Linear(embed_size,headsize,bias=False)
        self.query = nn.Linear(embed_size,headsize,bias=False)
        self.value = nn.Linear(embed_size,headsize,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        Batches, Time, Channels = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2,-1) * Channels**-0.5
        wei = wei.masked_fill(self.tril[:Time,:Time] == 0,float('-inf'))
        wei = F.softmax(wei,dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out

class Heads(nn.Module):
    def __init__(self,n_head,head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for i in range(n_head)])
        self.projection = nn.Linear(embed_size, embed_size)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        out = torch.cat([head(x) for head in self.heads],dim=-1)
        out = self.dropout(self.projection(out))
        return out

class FF_Layer(nn.Module):
    def __init__(self,embed_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embed_size,4*embed_size),
            nn.ReLU(),
            nn.Linear(4*embed_size,embed_size),
            nn.Dropout(dropout)
        )
    def forward(self,x):
        return self.net(x)
class BigramLM(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding_table = nn.Embedding(vocab_size,embed_size)
        self.position_embedding_table = nn.Embedding(block_size,embed_size)
        self.lm_head = nn.Linear(embed_size,vocab_size)
        self.blocks = nn.Sequential(*[trans_block(embed_size,heads = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(embed_size)
    def forward(self,idx,targets=None):
        Branch,Time = idx.shape

        token_embed = self.embedding_table(idx)
        position_embed = self.position_embedding_table(torch.arange(Time,device=device))
        added = token_embed + position_embed
        added = self.blocks(added)
        added = self.ln_f(added)
        logits = self.lm_head(added)

        if targets is None:
            loss = None
        else:
            Batch, Time, Channel = logits.shape
            logits = logits.view(Batch*Time,Channel)
            targets = targets.view(Batch*Time)
            loss = F.cross_entropy(logits,targets)
        return logits,loss
    def generate(self, idx, max_tokens):
        for i in range(max_tokens):
            idx_condition = idx[:, -block_size:]
            logits, loss = self(idx_condition)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

modelsomething = BigramLM()
model = modelsomething.to(device)
out = model(xb,yb)

idx = torch.zeros((1,1),dtype=torch.long,device = device)
print(decode(model.generate(idx,max_tokens = 100)[0].tolist()))

mutilated."marlcontainednow--oldtakenpeepingLeonardodid,lockspurposefulcries"Wellangular,unmade,mystery?XIIcrouched,swingfortuneexpressexpert."colourboneshere."familyanxietyintroduction.airdead-fish"What'sshriekcellarsmillionaire,pale--nevernews,""Hollowayuselessopenednations."Brandy!shared"areunawaresomewhatsave,Drivetillagreehope,pictures.thanklesspantryaffectionatefullbristlingEdmundsyet,'beatfelt,eventuallensvacated.poker.family.switchedNorthernSherlockconclusion.Presbury.handwriting."Sorrowpray,souls.paralyseddistinctbrother'sfather'sthat.sensemanifestationssister'sagainstsoup-platewheremid-Victorianbuckboards.mistress."Ronder."workmanlikeus?"TwelverefusesDoesreachingbristledbattlefieldplansgas,"morning?"begun


In [ ]:

optimizer = torch.optim.AdamW(model.parameters(),lr = 3e-4)
for i in range(5000):
    xb,yb = get_batch('train')
    _,loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print(loss.item())
print(loss.item())

In [ ]:

print(decode(model.generate(idx,max_tokens = 500)[0].tolist()))


braw, and I momess on the tache fand him innew, had, but the was reard pers.  Then can!  But He door at a weelf I not--the ly emprioret
lostreal-was eved.

"It wandys.

"I "THen Mr.

"Nined."

"You a last sund upon is on Hosk the inder."

You hadid had laske
you that the cundentle,
ible.  I goor
yould, your is I his loding-tand hushal, olles,
sperys thas I keald I have the har clxoCdmat whis nave tagrouginth pat land
mitters, and that the lorguse for Emes, Mr. ARobl0
wel-coall howf you am, what 
